In [1]:
%run 'header.ipynb'

/Users/sunny/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,50,51,52,53,54,55,56,58,59,60,61,62,63,64,65,67,68,69,70,71,72,73,74,76,77,78,79,80,81,82,83,85,86,87,88,89,90,91,92,94,95,96,97,98,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
## debug the format from ks_thai_all_raw.csv

df_raw = pd.read_csv('../data/morningstar/ks_thai_all_raw.csv',index_col=[0])
df = pd.DataFrame()
bad_tickers=[]
for ticker in df_raw.ticker.unique():
    try:
        dfs = df_raw[df_raw.ticker==ticker].dropna(axis=1,how='all')
        dfs.columns = [x.split('-')[0] for x in dfs.columns]
        df = df.append(dfs)
    except:
        bad_tickers.append(ticker)
    
    

/Users/sunny/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,23,24,25,26,27,28,29,30,32,33,34,35,36,37,38,39,41,42,43,44,45,46,47,48,50,51,52,53,54,55,56,58,59,60,61,62,63,64,65,67,68,69,70,71,72,73,74,76,77,78,79,80,81,82,83,85,86,87,88,89,90,91,92,94,95,96,97,98,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
len(df.ticker.unique()),len(bad_tickers)

(682, 2)

In [19]:
df.head()

,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,TTM,item,ticker
0,NaN,NaN,NaN,NaN,NaN,NaN,"2,795","2,560","2,813","3,695","4,059","4,092","4,235","3,698","3,936",NaN,"4,402",Revenue THB Mil,2S
1,NaN,NaN,NaN,NaN,NaN,NaN,9.3,8.1,6.9,5.4,4.9,5.6,5.0,5.3,15.2,NaN,12.8,Gross Margin %,2S
2,NaN,NaN,NaN,NaN,NaN,NaN,167,117,97,78,61,88,75,62,440,NaN,373,Operating Income THB Mil,2S
3,NaN,NaN,NaN,NaN,NaN,NaN,6.0,4.6,3.5,2.1,1.5,2.1,1.8,1.7,11.2,NaN,8.5,Operating Margin %,2S
4,NaN,NaN,NaN,NaN,NaN,NaN,113,92,71,65,53,68,42,34,343,NaN,293,Net Income THB Mil,2S


In [18]:
#split_candidate
set50_tickers = ['PTT', 'SCC', 'AOT', 'ADVANC', 'SCB', 'KBANK', 'CPALL', 'BDMS',
       'BBL', 'KTB', 'PTTEP', 'PTTGC', 'CPN', 'INTUCH', 'TRUE', 'MINT',
       'BH', 'CPF', 'TOP', 'LH', 'BTS', 'GLOW', 'TMB', 'IVL', 'DELTA',
       'BLA', 'HMPRO', 'IRPC', 'TU', 'EGCO', 'SCCC', 'DTAC', 'TASCO',
       'BEC', 'CENTEL', 'PS', 'M', 'CK', 'SAWAD', 'BA', 'ROBINS', 'TCAP',
       'BCP', 'TPIPL', 'TTW', 'BANPU', 'WHA', 'ITD', 'CBG', 'JAS']

#loop over tickers
split_df=pd.DataFrame()
for ticker in set50_tickers:
    dfs=df[df.ticker==ticker]['number_of_shares']
    ratio = (dfs/dfs.shift(1)).apply(lambda x: round(x,1))
    ratio= ratio[ratio > 1.2].to_frame()
    ratio['ticker']=ticker
    split_df=split_df.append(ratio)
    
split_df.to_csv('/Users/Sunny/Desktop/split_candidate.csv')

In [21]:
df['cash_ttm']=df['cash'].rolling(window=4).mean()
df['market_cap']=df['close_price']*df['number_of_shares']

tmp=df.groupby('ticker')['eps'].apply(lambda x: 1.0*np.alltrue(x>0))
tmp.name='always_profit'
df=df.join(tmp,on='ticker')

tmp=df[(df.always_profit > 0)].groupby('ticker').last()[['cash_ttm','number_of_shares','close_price','market_cap']]
tmp['ppc']=tmp['close_price']*tmp['number_of_shares']/tmp['cash_ttm']
tmp[tmp > -float("inf")].sort_values(by='ppc')

,cash_ttm,number_of_shares,close_price,market_cap,ppc
ticker,,,,,
WG,49659.1750,17.839416,150.00,2675.912409,0.053886
PRAKIT,14433.1675,55.666667,15.10,840.566667,0.058239
UBIS,4218.4150,239.777778,5.80,1390.711111,0.329676
SENA,8341.8175,1178.333333,3.94,4642.633333,0.556549
CHG,49519.6225,15286.000000,2.50,38215.000000,0.771714
TOPP,777.1050,6.004992,190.00,1140.948419,1.468204
BM,774.6325,399.000000,3.92,1564.080000,2.019125
EKH,1486.3450,684.500000,6.45,4415.025000,2.970390
GIFT,638.0650,285.750000,7.70,2200.275000,3.448356
